In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
import os
from collections import defaultdict
from datetime import datetime, timedelta
from typing import List

from src import auth_manager
from src import constants
from src.types.activity import Activity, DailyMetrics
from src.utils import datetime_now_est
from stravalib.client import Client


def add_missing_dates(
    activities: List[Activity], start_date: datetime, end_date: datetime
) -> List[Activity]:
    """
    Ensures that the list of activities includes placeholder activities for all dates
    between the start and end date.

    :param activities: List of Activity Pydantic models.
    :param start_date: The start date of the range.
    :param end_date: The end date of the range.
    :return: A list of Activity objects, with missing dates filled in as placeholder activities.
    """
    existing_dates = {activity.start_date_local.date() for activity in activities}
    total_days = (end_date.date() - start_date.date()).days + 1
    all_dates = {start_date.date() + timedelta(days=i) for i in range(total_days)}
    missing_dates = all_dates - existing_dates

    placeholders = [
        Activity(
            start_date=datetime.combine(date, datetime.min.time()),
            start_date_local=datetime.combine(date, datetime.min.time()),
        )
        for date in missing_dates
    ]
    return sorted(activities + placeholders, key=lambda x: x.start_date_local)


def aggregate_daily_metrics(activities: List[Activity]) -> List[DailyMetrics]:
    """
    Aggregates and transforms activity data to calculate daily and weekly metrics.

    :param activities: List of Activity Pydantic models containing activity data
    :return: A list of DailyMetrics objects with aggregated and transformed metrics
    """

    results = []
    activities_by_date = defaultdict(list)
    for activity in activities:
        activities_by_date[activity.start_date_local.date()].append(activity)

    for activity_date, daily_activities in activities_by_date.items():
        total_distance = sum(a.distance for a in daily_activities)
        total_elevation_gain = sum(a.total_elevation_gain for a in daily_activities)
        total_moving_time = sum(a.moving_time.total_seconds() for a in daily_activities)
        activity_count = len([a for a in daily_activities if a.id != -1])

        if total_distance > 0:
            pace_minutes_per_mile = (total_moving_time / 60) / (
                total_distance / constants.METERS_PER_MILE
            )
        else:
            pace_minutes_per_mile = None

        results.append(
            DailyMetrics(
                date=activity_date,
                day_of_week=activity_date.strftime("%a").lower(),
                week_of_year=activity_date.isocalendar()[1],
                year=activity_date.year,
                distance_in_miles=total_distance / constants.METERS_PER_MILE,
                elevation_gain_in_feet=total_elevation_gain * constants.FEET_PER_METER,
                moving_time_in_minutes=total_moving_time / 60,
                pace_minutes_per_mile=pace_minutes_per_mile,
                activity_count=activity_count,
            )
        )

    results = sorted(results, key=lambda x: x.date)
    first_week = min(item.week_of_year for item in results)
    results = [item for item in results if item.week_of_year != first_week]

    return results


strava_client = auth_manager.get_strava_client(os.environ["JAMIES_ATHLETE_ID"])
end_date = datetime_now_est()
start_date = end_date - timedelta(weeks=8)

all_strava_activities = strava_client.get_activities(
    after=start_date, before=end_date
)

# filter and convert to our Activity type
activities = [
    Activity(**activity.__dict__)
    for activity in all_strava_activities
    if activity.sport_type == "Run"
]

# add empty activities for missing dates
all_dates_activities = add_missing_dates(
    activities=activities, start_date=start_date, end_date=end_date
)

aggregate_daily_metrics(all_dates_activities)

[DailyMetrics(date=datetime.date(2024, 9, 16), day_of_week='mon', week_of_year=38, year=2024, distance_in_miles=0.0, elevation_gain_in_feet=0.0, moving_time_in_minutes=0.0, pace_minutes_per_mile=None, activity_count=0),
 DailyMetrics(date=datetime.date(2024, 9, 17), day_of_week='tue', week_of_year=38, year=2024, distance_in_miles=0.0, elevation_gain_in_feet=0.0, moving_time_in_minutes=0.0, pace_minutes_per_mile=None, activity_count=0),
 DailyMetrics(date=datetime.date(2024, 9, 18), day_of_week='wed', week_of_year=38, year=2024, distance_in_miles=0.0, elevation_gain_in_feet=0.0, moving_time_in_minutes=0.0, pace_minutes_per_mile=None, activity_count=0),
 DailyMetrics(date=datetime.date(2024, 9, 19), day_of_week='thu', week_of_year=38, year=2024, distance_in_miles=0.0, elevation_gain_in_feet=0.0, moving_time_in_minutes=0.0, pace_minutes_per_mile=None, activity_count=0),
 DailyMetrics(date=datetime.date(2024, 9, 20), day_of_week='fri', week_of_year=38, year=2024, distance_in_miles=0.0, ele